In [37]:
import pandas as pd
from datetime import datetime
import numpy as np

In [38]:
# Set end_period as a date object.
end_period = pd.to_datetime("2025-04-20").date()

In [39]:
# Path to the .xls file
file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\ml_goals.xls"

# Load the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame
df.head()


WARNING *** file size (18690011) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


,country,sezonul,datameci,orameci,etapa,txtechipa1,txtechipa2,scor1,scor2,scorp1,...,yellowa2,ballph,ballph1,ballph2,ballpa,ballpa1,ballpa2,stare,codechipa1,codechipa2
0,Ita1,25,2024-02-06,1945,14,Fiorentina,Inter,3,0,0,...,0,28,35,21,72,65,79,J,2008,2002
1,Mex1,25,2024-07-05,2345,1,Puebla,Santos Laguna,1,0,0,...,2,40,39,41,60,61,59,J,41017,41008
2,Mex1,25,2024-07-06,200,1,Queretaro,Tijuana de Caliente,1,2,0,...,0,37,33,41,63,67,59,J,41011,41006
3,Mex1,25,2024-07-06,410,1,Juarez,Atlas,2,2,1,...,0,55,66,44,45,34,56,J,41020,41002
4,Mex1,25,2024-07-07,0,1,San Luis,Club America,2,1,1,...,1,37,41,33,63,59,67,J,41019,41012


In [40]:
column_dict = {
    "country": "country",
    "league": "league",
    "sezonul": "season",
    "datameci": "date",
    "orameci": "ko_time",
    "etapa": "round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "scor1": "home_goals_ft",
    "scor2": "away_goals_ft",
    "scorp1": "home_goals_ht",
    "scorp2": "away_goals_ht",
    "place1": "home_team_place_total",
    "place1a": "home_team_place_home",
    "place2": "away_team_place_total",
    "place2d": "away_team_place_away",
    "cotaa": "home_odds",
    "cotae": "draw_odds",
    "cotad": "away_odds",
    # "cotao0": "",
    # "cotao1": "",
    "cotao": "over_25_odds",
    # "cotao3": "",
    # "cotao4": "",
    # "cotau0": "",
    # "cotau1": "",
    "cotau": "under_25_odds",
    # "cotau3": "",
    # "cotau4": "",
    # "gg": "",
    # "ng": "",
    "elohomeo": "elo_home",
    "eloawayo": "elo_away",
    "formah": "form_home",
    "formaa": "form_away",
    "suth": "shots_home",
    "suth1": "shots_home_1h",
    "suth2": "shots_home_2h",
    "suta": "shots_away",
    "suta1": "shots_away_1h",
    "suta2": "shots_away_2h",
    "sutht": "shots_on_target_home",
    "sutht1": "shots_on_target_home_1h",
    "sutht2": "shots_on_target_home_2h",
    "sutat": "shots_on_target_away",
    "sutat1": "shots_on_target_away_1h",
    "sutat2": "shots_on_target_away_2h",
    "corh": "corners_home",
    "corh1": "corners_home_1h",
    "corh2": "corners_home_2h",
    "cora": "corners_away",
    "cora1": "corners_away_1h",
    "cora2": "corners_away_2h",
    "foulsh": "fouls_home",
    "foulsh1": "fouls_home_1h",
    "foulsh2": "fouls_home_2h",
    "foulsa": "fouls_away",
    "foulsa1": "fouls_away_1h",
    "foulsa2": "fouls_away_2h",
    "yellowh": "yellow_cards_home",
    "yellowh1": "yellow_cards_home_1h",
    "yellowh2": "yellow_cards_home_2h",
    "yellowa": "yellow_cards_away",
    "yellowa1": "yellow_cards_away_1h",
    "yellowa2": "yellow_cards_away_2h",
    "ballph": "possession_home",
    "ballph1": "possession_home_1h",
    "ballph2": "possession_home_2h",
    "ballpa": "possession_away",
    "ballpa1": "possession_away_1h",
    "ballpa2": "possession_away_2h",
    "gsh": "goals_scored_total_home",
    "gch": "goals_conceded_total_home",
    "gsa": "goals_scored_total_away",
    "gca": "goals_conceded_total_away",
    # "stare": "",
    # "codechipa1": "",
    # "codechipa2": ""
}

df = df.rename(columns=column_dict).filter(items=column_dict.values())
data = df.copy()
#data

In [41]:
# Convert 'date' column to datetime object
data['date'] = pd.to_datetime(data['date'], format="%d/%m/%Y", errors='coerce')

# Order by date
data = data.sort_values(by='date')

# Filter out future dates (ensure data does not go beyond today)
today = datetime.today().date()
data = data[data['date'].dt.date <= end_period]

# Create a mask for matches that have been played (i.e. date is less than today)
played_mask = data['date'].dt.date < today

# Calculate home points for played matches only.
data.loc[played_mask, 'points_home'] = np.where(
    data.loc[played_mask, 'home_goals_ft'] > data.loc[played_mask, 'away_goals_ft'], 3,
    np.where(data.loc[played_mask, 'home_goals_ft'] == data.loc[played_mask, 'away_goals_ft'], 1, 0)
)

# Calculate away points for played matches only.
data.loc[played_mask, 'points_away'] = np.where(
    data.loc[played_mask, 'away_goals_ft'] > data.loc[played_mask, 'home_goals_ft'], 3,
    np.where(data.loc[played_mask, 'away_goals_ft'] == data.loc[played_mask, 'home_goals_ft'], 1, 0)
)

In [42]:
# # Assign points based on match results
# data["points_home"] = data.apply(lambda row: 3 if row["home_goals_ft"] > row["away_goals_ft"]
# else (1 if row["home_goals_ft"] == row["away_goals_ft"] else 0), axis=1)
#
# data["points_away"] = data.apply(lambda row: 3 if row["away_goals_ft"] > row["home_goals_ft"]
# else (1 if row["away_goals_ft"] == row["home_goals_ft"] else 0), axis=1)

# =============================================================================
# 1. Data Preparation: Build Team-Level Data (Home & Away)
# =============================================================================
# Prepare home records.
home_df = data[['country', 'season', 'date', 'home_team', 'away_team',
                'home_goals_ft', 'away_goals_ft', 'home_goals_ht', 'away_goals_ht',
                'shots_home', 'shots_home_1h', 'shots_home_2h',
                'shots_on_target_home', 'shots_on_target_home_1h', 'shots_on_target_home_2h',
                'corners_home', 'corners_home_1h', 'corners_home_2h']].copy()
home_df.rename(columns={
    'home_team': 'Team',
    'away_team': 'Opponent',
    'home_goals_ft': 'GoalsScored',
    'away_goals_ft': 'GoalsConceded',
    'home_goals_ht': 'FirstHalfGoalsScored',
    'away_goals_ht': 'FirstHalfGoalsConceded',
    'shots_home': 'Shots',
    'shots_home_1h': 'Shots_1h',
    'shots_home_2h': 'Shots_2h',
    'shots_on_target_home': 'ShotsOnTarget',
    'shots_on_target_home_1h': 'ShotsOnTarget_1h',
    'shots_on_target_home_2h': 'ShotsOnTarget_2h',
    'corners_home': 'Corners',
    'corners_home_1h': 'Corners_1h',
    'corners_home_2h': 'Corners_2h'
}, inplace=True)
home_df['is_home'] = 1

# Prepare away records.
away_df = data[['country', 'season', 'date', 'away_team', 'home_team',
                'away_goals_ft', 'home_goals_ft', 'away_goals_ht', 'home_goals_ht',
                'shots_away', 'shots_away_1h', 'shots_away_2h',
                'shots_on_target_away', 'shots_on_target_away_1h', 'shots_on_target_away_2h',
                'corners_away', 'corners_away_1h', 'corners_away_2h']].copy()
away_df.rename(columns={
    'away_team': 'Team',
    'home_team': 'Opponent',
    'away_goals_ft': 'GoalsScored',
    'home_goals_ft': 'GoalsConceded',
    'away_goals_ht': 'FirstHalfGoalsScored',
    'home_goals_ht': 'FirstHalfGoalsConceded',
    'shots_away': 'Shots',
    'shots_away_1h': 'Shots_1h',
    'shots_away_2h': 'Shots_2h',
    'shots_on_target_away': 'ShotsOnTarget',
    'shots_on_target_away_1h': 'ShotsOnTarget_1h',
    'shots_on_target_away_2h': 'ShotsOnTarget_2h',
    'corners_away': 'Corners',
    'corners_away_1h': 'Corners_1h',
    'corners_away_2h': 'Corners_2h'
}, inplace=True)
away_df['is_home'] = 0

# Combine both.
team_df = pd.concat([home_df, away_df], ignore_index=True)
team_df.sort_values(by=['country', 'season', 'Team', 'date'], inplace=True)

# Define rolling window sizes.
window_long = 5   # for long-term trends
window_short = 3  # for short-term momentum

# =============================================================================
# 2. Rolling Feature Computation Functions
# =============================================================================
def compute_slope(x):
    """Compute slope using simple linear regression."""
    if len(x) < 2:
        return np.nan
    xs = np.arange(len(x))
    return np.polyfit(xs, x, 1)[0]

def compute_rolling_features_metric(df_sub, full_col, first_half_col, prefix):
    """
    Compute rolling features for a given metric.
    Returns a DataFrame of new columns.
    """
    new_cols = {}
    # Full-match features.
    new_cols[f'{prefix}_Rolling_{full_col}_Mean'] = df_sub[full_col].rolling(window=window_long, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Rolling_{full_col}_Std']  = df_sub[full_col].rolling(window=window_long, min_periods=1).std().shift(1)
    new_cols[f'{prefix}_Rolling_{full_col}_Mean_Short'] = df_sub[full_col].rolling(window=window_short, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Momentum_{full_col}'] = new_cols[f'{prefix}_Rolling_{full_col}_Mean_Short'] - new_cols[f'{prefix}_Rolling_{full_col}_Mean']
    new_cols[f'{prefix}_Trend_Slope_{full_col}'] = df_sub[full_col].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)
    # First-half features.
    new_cols[f'{prefix}_Rolling_{first_half_col}_Mean'] = df_sub[first_half_col].rolling(window=window_long, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Rolling_{first_half_col}_Std']  = df_sub[first_half_col].rolling(window=window_long, min_periods=1).std().shift(1)
    new_cols[f'{prefix}_Rolling_{first_half_col}_Mean_Short'] = df_sub[first_half_col].rolling(window=window_short, min_periods=1).mean().shift(1)
    new_cols[f'{prefix}_Momentum_{first_half_col}'] = new_cols[f'{prefix}_Rolling_{first_half_col}_Mean_Short'] - new_cols[f'{prefix}_Rolling_{first_half_col}_Mean']
    new_cols[f'{prefix}_Trend_Slope_{first_half_col}'] = df_sub[first_half_col].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)
    return pd.DataFrame(new_cols, index=df_sub.index)

def add_rolling_features_split(group):
    """Compute overall, home-, and away-specific rolling features plus outcome percentages."""
    group = group.sort_values(by='date').reset_index(drop=True)

    # Overall features.
    overall_features = pd.concat([
        compute_rolling_features_metric(group, 'GoalsScored', 'FirstHalfGoalsScored', 'Overall'),
        compute_rolling_features_metric(group, 'Shots', 'Shots_1h', 'Overall'),
        compute_rolling_features_metric(group, 'Corners', 'Corners_1h', 'Overall'),
        compute_rolling_features_metric(group, 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Overall')
    ], axis=1)
    group = pd.concat([group, overall_features], axis=1)

    home_mask = group['is_home'] == 1
    away_mask = group['is_home'] == 0

    # Home-specific.
    if home_mask.any():
        home_feats = pd.concat([
            compute_rolling_features_metric(group.loc[home_mask], 'GoalsScored', 'FirstHalfGoalsScored', 'Home'),
            compute_rolling_features_metric(group.loc[home_mask], 'Shots', 'Shots_1h', 'Home'),
            compute_rolling_features_metric(group.loc[home_mask], 'Corners', 'Corners_1h', 'Home'),
            compute_rolling_features_metric(group.loc[home_mask], 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Home')
        ], axis=1)
        group.loc[home_mask, home_feats.columns] = home_feats

    # Away-specific.
    if away_mask.any():
        away_feats = pd.concat([
            compute_rolling_features_metric(group.loc[away_mask], 'GoalsScored', 'FirstHalfGoalsScored', 'Away'),
            compute_rolling_features_metric(group.loc[away_mask], 'Shots', 'Shots_1h', 'Away'),
            compute_rolling_features_metric(group.loc[away_mask], 'Corners', 'Corners_1h', 'Away'),
            compute_rolling_features_metric(group.loc[away_mask], 'ShotsOnTarget', 'ShotsOnTarget_1h', 'Away')
        ], axis=1)
        group.loc[away_mask, away_feats.columns] = away_feats

    # Additional outcome percentages for goals.
    thresh_dict = {}
    for thresh in [1.5, 2.5, 3.5]:
        thresh_dict[f'Overall_Percent_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        thresh_dict[f'Overall_Rolling5_Percent_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if home_mask.any():
            thresh_dict[f'Home_Percent_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            thresh_dict[f'Home_Rolling5_Percent_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if away_mask.any():
            thresh_dict[f'Away_Percent_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            thresh_dict[f'Away_Rolling5_Percent_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
    group = pd.concat([group, pd.DataFrame(thresh_dict, index=group.index)], axis=1)

    # Outcome percentages for goals.
    outcome_dict = {}
    for thresh in [0.5, 1.5, 2.5, 3.5]:
        outcome_dict[f'TeamPct_Over_{thresh}'] = group['GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        if home_mask.any():
            outcome_dict[f'Home_TeamPct_Over_{thresh}'] = group.loc[home_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        if away_mask.any():
            outcome_dict[f'Away_TeamPct_Over_{thresh}'] = group.loc[away_mask, 'GoalsScored'].gt(thresh).shift(1).expanding(min_periods=1).mean()
    group = pd.concat([group, pd.DataFrame(outcome_dict, index=group.index)], axis=1)

    # Outcome percentages for corners.
    corners_thresh = [3.5, 4.5, 5.5, 6.5]
    corners_dict = {}
    for thresh in corners_thresh:
        corners_dict[f'CornersPct_Over_{thresh}'] = group['Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
        corners_dict[f'CornersRolling5Pct_Over_{thresh}'] = group['Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if home_mask.any():
            corners_dict[f'Home_CornersPct_Over_{thresh}'] = group.loc[home_mask, 'Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            corners_dict[f'Home_CornersRolling5Pct_Over_{thresh}'] = group.loc[home_mask, 'Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
        if away_mask.any():
            corners_dict[f'Away_CornersPct_Over_{thresh}'] = group.loc[away_mask, 'Corners'].gt(thresh).shift(1).expanding(min_periods=1).mean()
            corners_dict[f'Away_CornersRolling5Pct_Over_{thresh}'] = group.loc[away_mask, 'Corners'].gt(thresh).shift(1).rolling(window=5, min_periods=1).mean()
    group = pd.concat([group, pd.DataFrame(corners_dict, index=group.index)], axis=1)

    return group

# Apply group-wise computations.
team_df = team_df.groupby(['country', 'season', 'Team'], group_keys=False).apply(add_rolling_features_split).reset_index(drop=True)
team_df = team_df.copy()  # ensure defragmentation

# =============================================================================
# 3. Compute Team-Level Corners Outcome Features (from Match Data)
# =============================================================================
# Build a match-level DataFrame for corners outcomes.
match_df = data.copy()
match_df['Total_Corners'] = match_df['corners_home'] + match_df['corners_away']
match_df.sort_values(by=['country', 'season', 'date'], inplace=True)

# Create a team perspective by combining home and away records.
home_matches = match_df[['country', 'season', 'date', 'home_team', 'Total_Corners']].copy()
home_matches.rename(columns={'home_team': 'Team'}, inplace=True)
away_matches = match_df[['country', 'season', 'date', 'away_team', 'Total_Corners']].copy()
away_matches.rename(columns={'away_team': 'Team'}, inplace=True)
team_corners_matches = pd.concat([home_matches, away_matches], ignore_index=True)
team_corners_matches.sort_values(by=['country', 'season', 'Team', 'date'], inplace=True)

# For thresholds 9.5, 10.5, and 11.5, compute season-level and rolling percentages.
for thr in [9.5, 10.5, 11.5]:
    indicator = f'Over_{thr}'
    team_corners_matches[indicator] = (team_corners_matches['Total_Corners'] > thr).astype(int)
    team_corners_matches[f'SeasonPct_{indicator}'] = team_corners_matches.groupby(
        ['country', 'season', 'Team']
    )[indicator].transform(lambda x: x.shift(1).expanding(min_periods=1).mean())
    team_corners_matches[f'Rolling5Pct_{indicator}'] = team_corners_matches.groupby(
        ['country', 'season', 'Team']
    )[indicator].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())

# Select only the keys and outcome columns for merging.
cols_to_merge = ['country', 'season', 'date', 'Team',
                 'SeasonPct_Over_9.5', 'Rolling5Pct_Over_9.5',
                 'SeasonPct_Over_10.5', 'Rolling5Pct_Over_10.5',
                 'SeasonPct_Over_11.5', 'Rolling5Pct_Over_11.5']

# Merge the corners outcome features into team_df.
team_df = team_df.merge(team_corners_matches[cols_to_merge],
                        on=['country', 'season', 'date', 'Team'],
                        how='left')

# =============================================================================
# 4. Process Home and Away Features for Match-Level Merging
# =============================================================================
# -- Home-Team Process --
home_subset = team_df[team_df['is_home'] == 1].copy()
home_subset.drop(columns=['Opponent'], inplace=True)
home_subset.rename(columns={'Team': 'home_team'}, inplace=True)
home_key = ['country', 'season', 'date', 'home_team', 'is_home']
# Include features starting with Overall_, Home_, SeasonPct_Over_, or Rolling5Pct_Over_
home_feats = [col for col in home_subset.columns if col not in home_key and
              (col.startswith("Overall_") or col.startswith("Home_") or
               col.startswith("SeasonPct_Over_") or col.startswith("Rolling5Pct_Over_"))]
home_features = home_subset[home_key + home_feats].copy()
def clean_home_name(col):
    return "home_" + (col[len("Home_"):] if col.startswith("Home_") else col)
home_features.rename(columns={col: clean_home_name(col) for col in home_feats}, inplace=True)

# -- Away-Team Process --
away_subset = team_df[team_df['is_home'] == 0].copy()
away_subset.drop(columns=['Opponent'], inplace=True)
away_subset.rename(columns={'Team': 'away_team'}, inplace=True)
away_key = ['country', 'season', 'date', 'away_team', 'is_home']
away_feats = [col for col in away_subset.columns if col not in away_key and
              (col.startswith("Overall_") or col.startswith("Away_") or
               col.startswith("SeasonPct_Over_") or col.startswith("Rolling5Pct_Over_"))]
away_features = away_subset[away_key + away_feats].copy()
def clean_away_name(col):
    return "away_" + (col[len("Away_"):] if col.startswith("Away_") else col)
away_features.rename(columns={col: clean_away_name(col) for col in away_feats}, inplace=True)

# =============================================================================
# 5. Merge Home and Away Features into the Match-Level DataFrame
# =============================================================================
# Start with the original match data.
match_merge_df = data.copy()
# Merge home features.
match_merge_df = match_merge_df.merge(home_features, on=['country', 'season', 'date', 'home_team'], how='left')
# Merge away features.
match_merge_df = match_merge_df.merge(away_features, on=['country', 'season', 'date', 'away_team'], how='left')

# (Optional) Display a sample.
#print(match_merge_df.head())


C:\Users\leere\AppData\Local\Temp\ipykernel_27688\3827831954.py:175: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  team_df = team_df.groupby(['country', 'season', 'Team'], group_keys=False).apply(add_rolling_features_split).reset_index(drop=True)


In [43]:
# match_merge_df_filter = match_merge_df[match_merge_df['away_team'] == "Chelsea"]
# match_merge_df_filter

In [44]:
## -----------------------------
# 1. Process Home-Team Features (with clean naming)
# -----------------------------
home_subset = team_df[team_df['is_home'] == 1].copy()
home_subset = home_subset.drop(columns=['Opponent'])
home_subset.rename(columns={'Team': 'home_team'}, inplace=True)

# Key columns that remain unchanged
home_key_cols = ['country', 'season', 'date', 'home_team', 'is_home']

# Update the feature column selection to include the merged outcome columns.
home_feature_cols = [col for col in home_subset.columns
                     if col not in home_key_cols and
                     (col.startswith("Overall_") or
                      col.startswith("Home_") or
                      col.startswith("SeasonPct_Over_") or
                      col.startswith("Rolling5Pct_Over_"))]

# Create a DataFrame with key columns and desired features
home_features = home_subset[home_key_cols + home_feature_cols].copy()

# Function to clean column names by removing any existing "Home_" prefix
def clean_home_name(col):
    if col.startswith("Home_"):
        col = col[len("Home_"):]
    return "home_" + col

# Build a renaming dictionary for home features
rename_mapping_home = {col: clean_home_name(col) for col in home_feature_cols}
home_features.rename(columns=rename_mapping_home, inplace=True)


# -----------------------------
# 2. Process Away-Team Features (with clean naming)
# -----------------------------
away_subset = team_df[team_df['is_home'] == 0].copy()
away_subset = away_subset.drop(columns=['Opponent'])
away_subset.rename(columns={'Team': 'away_team'}, inplace=True)

# Key columns that remain unchanged
away_key_cols = ['country', 'season', 'date', 'away_team', 'is_home']

# Update the feature column selection to include the merged outcome columns.
away_feature_cols = [col for col in away_subset.columns
                     if col not in away_key_cols and
                     (col.startswith("Overall_") or
                      col.startswith("Away_") or
                      col.startswith("SeasonPct_Over_") or
                      col.startswith("Rolling5Pct_Over_"))]

# Create a DataFrame with key columns and desired features
away_features = away_subset[away_key_cols + away_feature_cols].copy()

# Function to clean column names by removing any existing "Away_" prefix
def clean_away_name(col):
    if col.startswith("Away_"):
        col = col[len("Away_"):]
    return "away_" + col

# Build a renaming dictionary for away features
rename_mapping_away = {col: clean_away_name(col) for col in away_feature_cols}
away_features.rename(columns=rename_mapping_away, inplace=True)


# -----------------------------
# 3. Merge Processed Home- and Away-Team Features Back into the Match-Level DataFrame
# -----------------------------
# Start with your original match-level data
match_df = data.copy()

# Merge home features on the common keys: country, season, date, and home_team.
match_df = match_df.merge(home_features, on=['country', 'season', 'date', 'home_team'], how='left')

#Merge away features on the common keys: country, season, date, and away_team.
match_df = match_df.merge(away_features, on=['country', 'season', 'date', 'away_team'], how='left')

# # match_df now contains cleanly named columns such as "home_Rolling_GoalsScored_Mean" along with your corners outcome features.
# print(match_df.head())


In [45]:
# team_filter2 = match_df[(match_df["home_team"]=="Chelsea") | (match_df["away_team"]=="Chelsea")]
# team_filter2

In [46]:
#match_df

In [47]:
features = [
    # 'Unnamed: 0',
    # 'country',
    # 'season',
    # 'date',
    # 'ko_time',
    'round',
    # 'home_team',
    # 'away_team',
    # 'home_goals_ft',
    # 'away_goals_ft',
    # 'home_goals_ht',
    # 'away_goals_ht',
    'home_team_place_total',
    'home_team_place_home',
    'away_team_place_total',
    'away_team_place_away',
    'home_odds',
    'draw_odds',
    'away_odds',
    'over_25_odds',
    'under_25_odds',
    'elo_home',
    'elo_away',
    'form_home',
    'form_away',
    # 'shots_home',
    # 'shots_home_1h',
    # 'shots_home_2h',
    # 'shots_away',
    # 'shots_away_1h',
    # 'shots_away_2h',
    # 'shots_on_target_home',
    # 'shots_on_target_home_1h',
    # 'shots_on_target_home_2h',
    # 'shots_on_target_away',
    # 'shots_on_target_away_1h',
    # 'shots_on_target_away_2h',
    # 'corners_home',
    # 'corners_home_1h',
    # 'corners_home_2h',
    # 'corners_away',
    # 'corners_away_1h',
    # 'corners_away_2h',
    # 'fouls_home',
    # 'fouls_home_1h',
    # 'fouls_home_2h',
    # 'fouls_away',
    # 'fouls_away_1h',
    # 'fouls_away_2h',
    # 'yellow_cards_home',
    # 'yellow_cards_home_1h',
    # 'yellow_cards_home_2h',
    # 'yellow_cards_away',
    # 'yellow_cards_away_1h',
    # 'yellow_cards_away_2h',
    # 'possession_home',
    # 'possession_home_1h',
    # 'possession_home_2h',
    # 'possession_away',
    # 'possession_away_1h',
    # 'possession_away_2h',
    # 'goals_scored_total_home',
    # 'goals_conceded_total_home',
    # 'goals_scored_total_away',
    # 'goals_conceded_total_away',
    # 'points_home',
    # 'points_away',
    # 'is_home_x',
    'home_Overall_Rolling_GoalsScored_Mean',
    'home_Overall_Rolling_GoalsScored_Std',
    'home_Overall_Rolling_GoalsScored_Mean_Short',
    'home_Overall_Momentum_GoalsScored',
    'home_Overall_Trend_Slope_GoalsScored',
    'home_Overall_Rolling_FirstHalfGoalsScored_Mean',
    'home_Overall_Rolling_FirstHalfGoalsScored_Std',
    'home_Overall_Rolling_FirstHalfGoalsScored_Mean_Short',
    'home_Overall_Momentum_FirstHalfGoalsScored',
    'home_Overall_Trend_Slope_FirstHalfGoalsScored',
    'home_Overall_Rolling_Shots_Mean',
    'home_Overall_Rolling_Shots_Std',
    'home_Overall_Rolling_Shots_Mean_Short',
    'home_Overall_Momentum_Shots',
    'home_Overall_Trend_Slope_Shots',
    'home_Overall_Rolling_Shots_1h_Mean',
    'home_Overall_Rolling_Shots_1h_Std',
    'home_Overall_Rolling_Shots_1h_Mean_Short',
    'home_Overall_Momentum_Shots_1h',
    'home_Overall_Trend_Slope_Shots_1h',
    'home_Overall_Rolling_Corners_Mean',
    'home_Overall_Rolling_Corners_Std',
    'home_Overall_Rolling_Corners_Mean_Short',
    'home_Overall_Momentum_Corners',
    'home_Overall_Trend_Slope_Corners',
    'home_Overall_Rolling_Corners_1h_Mean',
    'home_Overall_Rolling_Corners_1h_Std',
    'home_Overall_Rolling_Corners_1h_Mean_Short',
    'home_Overall_Momentum_Corners_1h',
    'home_Overall_Trend_Slope_Corners_1h',
    'home_Overall_Rolling_ShotsOnTarget_Mean',
    'home_Overall_Rolling_ShotsOnTarget_Std',
    'home_Overall_Rolling_ShotsOnTarget_Mean_Short',
    'home_Overall_Momentum_ShotsOnTarget',
    'home_Overall_Trend_Slope_ShotsOnTarget',
    'home_Overall_Rolling_ShotsOnTarget_1h_Mean',
    'home_Overall_Rolling_ShotsOnTarget_1h_Std',
    'home_Overall_Rolling_ShotsOnTarget_1h_Mean_Short',
    'home_Overall_Momentum_ShotsOnTarget_1h',
    'home_Overall_Trend_Slope_ShotsOnTarget_1h',
    'home_Rolling_GoalsScored_Mean',
    'home_Rolling_GoalsScored_Std',
    'home_Rolling_GoalsScored_Mean_Short',
    'home_Momentum_GoalsScored',
    'home_Trend_Slope_GoalsScored',
    'home_Rolling_FirstHalfGoalsScored_Mean',
    'home_Rolling_FirstHalfGoalsScored_Std',
    'home_Rolling_FirstHalfGoalsScored_Mean_Short',
    'home_Momentum_FirstHalfGoalsScored',
    'home_Trend_Slope_FirstHalfGoalsScored',
    'home_Rolling_Shots_Mean',
    'home_Rolling_Shots_Std',
    'home_Rolling_Shots_Mean_Short',
    'home_Momentum_Shots',
    'home_Trend_Slope_Shots',
    'home_Rolling_Shots_1h_Mean',
    'home_Rolling_Shots_1h_Std',
    'home_Rolling_Shots_1h_Mean_Short',
    'home_Momentum_Shots_1h',
    'home_Trend_Slope_Shots_1h',
    'home_Rolling_Corners_Mean',
    'home_Rolling_Corners_Std',
    'home_Rolling_Corners_Mean_Short',
    'home_Momentum_Corners',
    'home_Trend_Slope_Corners',
    'home_Rolling_Corners_1h_Mean',
    'home_Rolling_Corners_1h_Std',
    'home_Rolling_Corners_1h_Mean_Short',
    'home_Momentum_Corners_1h',
    'home_Trend_Slope_Corners_1h',
    'home_Rolling_ShotsOnTarget_Mean',
    'home_Rolling_ShotsOnTarget_Std',
    'home_Rolling_ShotsOnTarget_Mean_Short',
    'home_Momentum_ShotsOnTarget',
    'home_Trend_Slope_ShotsOnTarget',
    'home_Rolling_ShotsOnTarget_1h_Mean',
    'home_Rolling_ShotsOnTarget_1h_Std',
    'home_Rolling_ShotsOnTarget_1h_Mean_Short',
    'home_Momentum_ShotsOnTarget_1h',
    'home_Trend_Slope_ShotsOnTarget_1h',
    'home_Overall_Percent_Over_1.5',
    'home_Overall_Rolling5_Percent_Over_1.5',
    'home_Percent_Over_1.5',
    'home_Rolling5_Percent_Over_1.5',
    'home_Overall_Percent_Over_2.5',
    'home_Overall_Rolling5_Percent_Over_2.5',
    'home_Percent_Over_2.5',
    'home_Rolling5_Percent_Over_2.5',
    'home_Overall_Percent_Over_3.5',
    'home_Overall_Rolling5_Percent_Over_3.5',
    'home_Percent_Over_3.5',
    'home_Rolling5_Percent_Over_3.5',
    'home_TeamPct_Over_0.5',
    'home_TeamPct_Over_1.5',
    'home_TeamPct_Over_2.5',
    'home_TeamPct_Over_3.5',
    'home_CornersPct_Over_3.5',
    'home_CornersRolling5Pct_Over_3.5',
    'home_CornersPct_Over_4.5',
    'home_CornersRolling5Pct_Over_4.5',
    'home_CornersPct_Over_5.5',
    'home_CornersRolling5Pct_Over_5.5',
    'home_CornersPct_Over_6.5',
    'home_CornersRolling5Pct_Over_6.5',
    'home_SeasonPct_Over_9.5',
    'home_Rolling5Pct_Over_9.5',
    'home_SeasonPct_Over_10.5',
    'home_Rolling5Pct_Over_10.5',
    'home_SeasonPct_Over_11.5',
    'home_Rolling5Pct_Over_11.5',
    # 'is_home_y',
    'away_Overall_Rolling_GoalsScored_Mean',
    'away_Overall_Rolling_GoalsScored_Std',
    'away_Overall_Rolling_GoalsScored_Mean_Short',
    'away_Overall_Momentum_GoalsScored',
    'away_Overall_Trend_Slope_GoalsScored',
    'away_Overall_Rolling_FirstHalfGoalsScored_Mean',
    'away_Overall_Rolling_FirstHalfGoalsScored_Std',
    'away_Overall_Rolling_FirstHalfGoalsScored_Mean_Short',
    'away_Overall_Momentum_FirstHalfGoalsScored',
    'away_Overall_Trend_Slope_FirstHalfGoalsScored',
    'away_Overall_Rolling_Shots_Mean',
    'away_Overall_Rolling_Shots_Std',
    'away_Overall_Rolling_Shots_Mean_Short',
    'away_Overall_Momentum_Shots',
    'away_Overall_Trend_Slope_Shots',
    'away_Overall_Rolling_Shots_1h_Mean',
    'away_Overall_Rolling_Shots_1h_Std',
    'away_Overall_Rolling_Shots_1h_Mean_Short',
    'away_Overall_Momentum_Shots_1h',
    'away_Overall_Trend_Slope_Shots_1h',
    'away_Overall_Rolling_Corners_Mean',
    'away_Overall_Rolling_Corners_Std',
    'away_Overall_Rolling_Corners_Mean_Short',
    'away_Overall_Momentum_Corners',
    'away_Overall_Trend_Slope_Corners',
    'away_Overall_Rolling_Corners_1h_Mean',
    'away_Overall_Rolling_Corners_1h_Std',
    'away_Overall_Rolling_Corners_1h_Mean_Short',
    'away_Overall_Momentum_Corners_1h',
    'away_Overall_Trend_Slope_Corners_1h',
    'away_Overall_Rolling_ShotsOnTarget_Mean',
    'away_Overall_Rolling_ShotsOnTarget_Std',
    'away_Overall_Rolling_ShotsOnTarget_Mean_Short',
    'away_Overall_Momentum_ShotsOnTarget',
    'away_Overall_Trend_Slope_ShotsOnTarget',
    'away_Overall_Rolling_ShotsOnTarget_1h_Mean',
    'away_Overall_Rolling_ShotsOnTarget_1h_Std',
    'away_Overall_Rolling_ShotsOnTarget_1h_Mean_Short',
    'away_Overall_Momentum_ShotsOnTarget_1h',
    'away_Overall_Trend_Slope_ShotsOnTarget_1h',
    'away_Rolling_GoalsScored_Mean',
    'away_Rolling_GoalsScored_Std',
    'away_Rolling_GoalsScored_Mean_Short',
    'away_Momentum_GoalsScored',
    'away_Trend_Slope_GoalsScored',
    'away_Rolling_FirstHalfGoalsScored_Mean',
    'away_Rolling_FirstHalfGoalsScored_Std',
    'away_Rolling_FirstHalfGoalsScored_Mean_Short',
    'away_Momentum_FirstHalfGoalsScored',
    'away_Trend_Slope_FirstHalfGoalsScored',
    'away_Rolling_Shots_Mean',
    'away_Rolling_Shots_Std',
    'away_Rolling_Shots_Mean_Short',
    'away_Momentum_Shots',
    'away_Trend_Slope_Shots',
    'away_Rolling_Shots_1h_Mean',
    'away_Rolling_Shots_1h_Std',
    'away_Rolling_Shots_1h_Mean_Short',
    'away_Momentum_Shots_1h',
    'away_Trend_Slope_Shots_1h',
    'away_Rolling_Corners_Mean',
    'away_Rolling_Corners_Std',
    'away_Rolling_Corners_Mean_Short',
    'away_Momentum_Corners',
    'away_Trend_Slope_Corners',
    'away_Rolling_Corners_1h_Mean',
    'away_Rolling_Corners_1h_Std',
    'away_Rolling_Corners_1h_Mean_Short',
    'away_Momentum_Corners_1h',
    'away_Trend_Slope_Corners_1h',
    'away_Rolling_ShotsOnTarget_Mean',
    'away_Rolling_ShotsOnTarget_Std',
    'away_Rolling_ShotsOnTarget_Mean_Short',
    'away_Momentum_ShotsOnTarget',
    'away_Trend_Slope_ShotsOnTarget',
    'away_Rolling_ShotsOnTarget_1h_Mean',
    'away_Rolling_ShotsOnTarget_1h_Std',
    'away_Rolling_ShotsOnTarget_1h_Mean_Short',
    'away_Momentum_ShotsOnTarget_1h',
    'away_Trend_Slope_ShotsOnTarget_1h',
    'away_Overall_Percent_Over_1.5',
    'away_Overall_Rolling5_Percent_Over_1.5',
    'away_Percent_Over_1.5',
    'away_Rolling5_Percent_Over_1.5',
    'away_Overall_Percent_Over_2.5',
    'away_Overall_Rolling5_Percent_Over_2.5',
    'away_Percent_Over_2.5',
    'away_Rolling5_Percent_Over_2.5',
    'away_Overall_Percent_Over_3.5',
    'away_Overall_Rolling5_Percent_Over_3.5',
    'away_Percent_Over_3.5',
    'away_Rolling5_Percent_Over_3.5',
    'away_TeamPct_Over_0.5',
    'away_TeamPct_Over_1.5',
    'away_TeamPct_Over_2.5',
    'away_TeamPct_Over_3.5',
    'away_CornersPct_Over_3.5',
    'away_CornersRolling5Pct_Over_3.5',
    'away_CornersPct_Over_4.5',
    'away_CornersRolling5Pct_Over_4.5',
    'away_CornersPct_Over_5.5',
    'away_CornersRolling5Pct_Over_5.5',
    'away_CornersPct_Over_6.5',
    'away_CornersRolling5Pct_Over_6.5',
    'away_SeasonPct_Over_9.5',
    'away_Rolling5Pct_Over_9.5',
    'away_SeasonPct_Over_10.5',
    'away_Rolling5Pct_Over_10.5',
    'away_SeasonPct_Over_11.5',
    'away_Rolling5Pct_Over_11.5'
]

In [48]:
filtered_data = match_df[(match_df['date'].dt.date >= today) & (match_df['date'].dt.date <= end_period)]
filtered_data

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_5.5,away_CornersRolling5Pct_Over_5.5,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5
10461,Bra1,25,2025-04-17,2300,4,Botafogo RJ,Sao Paulo,0,0,0,...,0.000000,0.0,0.000000,0.0,0.666667,0.666667,0.333333,0.333333,0.333333,0.333333
10462,Bra1,25,2025-04-17,130,4,Vitoria,Fortaleza,0,0,0,...,0.000000,0.0,0.000000,0.0,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000
10463,Cro1,25,2025-04-17,1700,30,Sibenik,Dinamo Zagreb,0,0,0,...,0.500000,0.6,0.428571,0.4,0.448276,0.800000,0.448276,0.800000,0.413793,0.800000
10464,Bra1,25,2025-04-17,130,4,Flamengo,Juventude RS,0,0,0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10465,Eng4,25,2025-04-17,2000,43,Bradford,Notts,0,0,0,...,0.333333,0.4,0.238095,0.4,0.404762,0.000000,0.309524,0.000000,0.214286,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10762,Ger3,25,2025-04-20,1230,34,Saarbrucken,Dresden,0,0,0,...,0.562500,0.6,0.375000,0.4,0.696970,1.000000,0.575758,0.800000,0.515152,0.600000
10763,Ned1,25,2025-04-20,1115,31,Utrecht,Amsterdam,0,0,0,...,0.200000,0.2,0.200000,0.2,0.241379,0.200000,0.172414,0.200000,0.172414,0.200000
10764,Ned2,25,2025-04-20,1330,35,Den Haag,Jong Eindhoven,0,0,0,...,0.235294,0.4,0.058824,0.0,0.705882,0.400000,0.441176,0.200000,0.352941,0.200000
10765,Ger3,25,2025-04-20,1530,34,Hannover II,Mannheim,0,0,0,...,0.375000,0.2,0.250000,0.2,0.636364,0.400000,0.484848,0.400000,0.393939,0.200000


In [49]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from joblib import load

# Assume that:
# 1. 'filtered_data' is your DataFrame filtered by dates.
# 2. 'features' is a list of your feature column names.
feature_columns = features  # e.g. features = ['feat1', 'feat2', 'feat3']

# Directory where the models are saved.
model_dir = r"C:\Users\leere\PycharmProjects\Football_ML3\Goals\Goals_v3\path\to\save\models"

# List to store prediction DataFrames per country.
predictions_list = []

# Iterate over each unique country in filtered_data.
for country in filtered_data['country'].unique():
    # Filter the dataframe for the current country.
    country_df = filtered_data[filtered_data['country'] == country].copy()

    # Construct the model filename.
    # Expected filename pattern: trained_model_('CountryName',).pkl
    model_filename = os.path.join(model_dir, f"trained_model_({repr(country)},).pkl")

    # If the model file does not exist, skip this country.
    if not os.path.exists(model_filename):
        print(f"Model file not found for country: {country}. Skipping.")
        continue

    # Load the model file using joblib.
    try:
        model_data = load(model_filename)
    except Exception as e:
        print(f"Error loading model file for {country}: {e}")
        continue

    # Debug: print the type of the loaded object.
    print(f"Country: {country} | Loaded model_data type: {type(model_data)}")

    # In your case the pickle files contain a dictionary.
    if isinstance(model_data, dict):
        # Extract the pipeline (i.e. your model) and threshold.
        pipeline = model_data.get('pipeline')
        threshold = model_data.get('threshold', 0.5)
        # Optionally you can also see the 'league' or 'smote_level' if needed:
        # league_info = model_data.get('league')
        # smote_level = model_data.get('smote_level')
        if pipeline is None:
            print(f"No pipeline found in model file for country: {country}. Skipping.")
            continue
    else:
        print(f"Model data for country {country} is not in the expected dictionary format. Skipping.")
        continue

    # Prepare the features for prediction.
    X = country_df[feature_columns]

    # Try to generate predictions.
    try:
        # If the model has predict_proba, use it to create probability predictions.
        if hasattr(pipeline, "predict_proba"):
            proba = pipeline.predict_proba(X)[:, 1]  # assuming binary classification; take the positive class probability.
            preds = (proba >= threshold).astype(int)
            country_df['prediction_proba'] = proba
        else:
            preds = pipeline.predict(X)
    except Exception as e:
        print(f"Error making predictions for country {country}: {e}")
        continue

    # Add the prediction column to the DataFrame.
    country_df['prediction'] = preds
    predictions_list.append(country_df)

# Merge predictions from all countries into a single DataFrame.
if predictions_list:
    all_predictions = pd.concat(predictions_list, ignore_index=True)
    print(all_predictions.head())
else:
    print("No predictions to merge.")


Model file not found for country: Bra1. Skipping.
Model file not found for country: Cro1. Skipping.
Country: Eng4 | Loaded model_data type: <class 'dict'>
Country: Aut1 | Loaded model_data type: <class 'dict'>
Model file not found for country: Swi2. Skipping.
Model file not found for country: Sco2. Skipping.
Country: Ned2 | Loaded model_data type: <class 'dict'>
Model file not found for country: Ire1. Skipping.
Model file not found for country: Bul1. Skipping.
Country: Aus2 | Loaded model_data type: <class 'dict'>
Model file not found for country: Chl1. Skipping.
Model file not found for country: Tur2. Skipping.
Country: Tur1 | Loaded model_data type: <class 'dict'>
Model file not found for country: Spa2. Skipping.
Country: Spa1 | Loaded model_data type: <class 'dict'>
Country: Por1 | Loaded model_data type: <class 'dict'>
Country: Swe1 | Loaded model_data type: <class 'dict'>
Model file not found for country: Arg1. Skipping.
Model file not found for country: Fra1. Skipping.
Model file

In [50]:
all_predictions

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5,prediction_proba,prediction
0,Eng4,25,2025-04-17,2000,43,Bradford,Notts,0,0,0,...,0.238095,0.4,0.404762,0.0,0.309524,0.0,0.214286,0.0,0.312200,0
1,Eng4,25,2025-04-18,1500,43,Walsall,Harrogate,0,0,0,...,0.190476,0.2,0.571429,0.2,0.547619,0.2,0.452381,0.2,0.426418,0
2,Eng4,25,2025-04-18,1500,43,Tranmere,Doncaster,0,0,0,...,0.333333,0.6,0.523810,0.4,0.476190,0.4,0.380952,0.4,0.341217,0
3,Eng4,25,2025-04-18,1500,43,Salford,Accrington,0,0,0,...,0.142857,0.0,0.452381,0.4,0.309524,0.0,0.095238,0.0,0.283833,0
4,Eng4,25,2025-04-18,1500,43,Milton Keynes,Newport County,0,0,0,...,0.000000,0.0,0.571429,0.4,0.404762,0.4,0.285714,0.2,0.483515,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Hun1,25,2025-04-19,1930,28,Paksi,Puskas Akademia,0,0,0,...,0.076923,0.2,0.440000,0.6,0.320000,0.6,0.320000,0.6,0.656248,1
120,Hun1,25,2025-04-19,1215,28,Debreceni,Zalaegerszeg,0,0,0,...,0.153846,0.2,0.518519,0.4,0.444444,0.4,0.333333,0.4,0.439481,0
121,Hun1,25,2025-04-19,1515,28,Diosgyori,Gyori ETO,0,0,0,...,0.307692,0.2,0.370370,0.4,0.296296,0.2,0.259259,0.2,0.427861,0
122,Hun1,25,2025-04-20,1215,28,Fehervar,MTK Budapest,0,0,0,...,0.071429,0.0,0.444444,0.4,0.333333,0.2,0.259259,0.2,0.622887,1


In [51]:
positive_pred = all_predictions[all_predictions['prediction']==1]
positive_pred

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5,prediction_proba,prediction
12,Aut1,25,2025-04-17,1035,27,Brisbane,Western United,0,0,0,...,0.500000,0.4,0.608696,0.800000,0.521739,0.600000,0.478261,0.600000,0.546816,1
13,Aut1,25,2025-04-18,1035,27,Adelaide,Wellington,0,0,0,...,0.000000,0.0,0.521739,0.600000,0.434783,0.400000,0.304348,0.400000,0.554357,1
14,Aut1,25,2025-04-19,800,27,Melbourne Victory,Auckland FC,0,0,0,...,0.090909,0.0,0.565217,0.400000,0.478261,0.400000,0.347826,0.200000,0.546945,1
15,Aut1,25,2025-04-19,1035,27,W. Sydney,Melbourne City,0,0,0,...,0.333333,0.4,0.304348,0.200000,0.217391,0.200000,0.130435,0.200000,0.522525,1
16,Aut1,25,2025-04-20,800,27,Central Coast,Macarthur FC,0,0,0,...,0.333333,0.4,0.695652,0.600000,0.608696,0.400000,0.478261,0.400000,0.545886,1
17,Aut1,25,2025-04-20,600,27,Sydney,Newcastle Jets,0,0,0,...,0.272727,0.4,0.782609,0.800000,0.652174,0.800000,0.565217,0.800000,0.554415,1
20,Ned2,25,2025-04-18,1900,35,Emmen,Jong Alkmaar,0,0,0,...,0.235294,0.4,0.558824,0.400000,0.529412,0.400000,0.529412,0.400000,0.586130,1
26,Ned2,25,2025-04-20,1545,35,Volendam,Excelsior,0,0,0,...,0.352941,0.4,0.558824,0.200000,0.441176,0.200000,0.352941,0.200000,0.619648,1
27,Ned2,25,2025-04-20,1330,35,Den Haag,Jong Eindhoven,0,0,0,...,0.058824,0.0,0.705882,0.400000,0.441176,0.200000,0.352941,0.200000,0.627950,1
28,Aus2,25,2025-04-18,1700,24,Sturm Graz (Am),First Vienna,0,0,0,...,0.272727,0.2,0.565217,0.600000,0.347826,0.600000,0.347826,0.600000,0.627156,1


In [57]:
import function_library as fl
import importlib
importlib.reload(fl)

positive_pred = fl.team_name_map(positive_pred)

C:\Users\leere\PycharmProjects\Football_ML3\function_library.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[home_col] = dataframe[home_col].map(mapping_dict).fillna(dataframe[home_col])
C:\Users\leere\PycharmProjects\Football_ML3\function_library.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[away_col] = dataframe[away_col].map(mapping_dict).fillna(dataframe[away_col])


In [58]:
positive_pred

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_CornersPct_Over_6.5,away_CornersRolling5Pct_Over_6.5,away_SeasonPct_Over_9.5,away_Rolling5Pct_Over_9.5,away_SeasonPct_Over_10.5,away_Rolling5Pct_Over_10.5,away_SeasonPct_Over_11.5,away_Rolling5Pct_Over_11.5,prediction_proba,prediction
12,Aut1,25,2025-04-17,1035,27,Brisbane Roar,Western United,0,0,0,...,0.500000,0.4,0.608696,0.800000,0.521739,0.600000,0.478261,0.600000,0.546816,1
13,Aut1,25,2025-04-18,1035,27,Adelaide United,Wellington Phoenix,0,0,0,...,0.000000,0.0,0.521739,0.600000,0.434783,0.400000,0.304348,0.400000,0.554357,1
14,Aut1,25,2025-04-19,800,27,Melbourne Victory,Auckland FC,0,0,0,...,0.090909,0.0,0.565217,0.400000,0.478261,0.400000,0.347826,0.200000,0.546945,1
15,Aut1,25,2025-04-19,1035,27,Western Sydney Wanderers,Melbourne City,0,0,0,...,0.333333,0.4,0.304348,0.200000,0.217391,0.200000,0.130435,0.200000,0.522525,1
16,Aut1,25,2025-04-20,800,27,Central Coast Mariners,Macarthur FC,0,0,0,...,0.333333,0.4,0.695652,0.600000,0.608696,0.400000,0.478261,0.400000,0.545886,1
17,Aut1,25,2025-04-20,600,27,Sydney FC,Newcastle Jets,0,0,0,...,0.272727,0.4,0.782609,0.800000,0.652174,0.800000,0.565217,0.800000,0.554415,1
20,Ned2,25,2025-04-18,1900,35,Emmen,Jong AZ Alkmaar,0,0,0,...,0.235294,0.4,0.558824,0.400000,0.529412,0.400000,0.529412,0.400000,0.586130,1
26,Ned2,25,2025-04-20,1545,35,FC Volendam,Excelsior,0,0,0,...,0.352941,0.4,0.558824,0.200000,0.441176,0.200000,0.352941,0.200000,0.619648,1
27,Ned2,25,2025-04-20,1330,35,ADO Den Haag,Jong PSV Eindhoven,0,0,0,...,0.058824,0.0,0.705882,0.400000,0.441176,0.200000,0.352941,0.200000,0.627950,1
28,Aus2,25,2025-04-18,1700,24,SK Sturm Graz II,First Vienna Fc 1894,0,0,0,...,0.272727,0.2,0.565217,0.600000,0.347826,0.600000,0.347826,0.600000,0.627156,1


In [59]:
fl.create_import_file(positive_pred, r"C:\Users\leere\OneDrive\Desktop\IMPORTS\BACK_O25_ALL_IMPORT.csv", provider="lay_all_u25", market_name="Over/Under 2.5 Goals", selection_name="Over 2.5 Goals")



File created and saved successfully at: C:\Users\leere\OneDrive\Desktop\IMPORTS\BACK_O25_ALL_IMPORT.csv
